In [1]:
import json
import re
import os
import calendar

import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go


pd.set_option('display.max_rows', 500)
months = calendar.month_abbr[1:]

In [49]:
file_path = r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly\embc2013.xlsx'
df = pd.read_excel(file_path,  sheet_name='May', header=None)

In [50]:
df[0] = df[0].fillna('')
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,"Date: May 8, 2022 Start: 07:15 Finish: 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tide State: low Tide Movement: falling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Observers: Judith Vetsch, Patrick MacNamara, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Brian Storey, Christopher Di Corrado, Amenda N...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Species: 92,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,House Finch,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3,NaN,NaN,NaN,NaN,1,NaN,2,NaN
97,Pine Siskin,43,NaN,NaN,NaN,NaN,NaN,1,1,1,...,NaN,35,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
98,American Goldfinch,12,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
99,,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW,NaN


In [6]:
info_1 = df.iloc[0,0]
info_2 = df.iloc[1,0]
info_3 = df.iloc[2,0]
info_4 = df.iloc[3,0]
info = info_1 + '  ' + info_2 + '  ' + info_3 + '  ' + info_4 + '  '


In [7]:
observers_match = re.search(r'(Observers: +)([\w -,]+\w+)(   *)', info)
observers = observers_match.group(2)
observers

'Dave Lassmann, Judith Vetsch, Patrick MacNamara, Chris Murrell, Grant Danielson, Sophie Vielfaure, Karl Ricker, Tiffany Brunke, Kyle Kulas, Chris Dale'

In [8]:
equipment_match = re.search(r'(Equipment: +)([\w -]+\w+)(   *)', info)
equipment = equipment_match.group(2)
equipment

'scope - binoculars only'

In [9]:
tide_movement_match = re.search(r'(Tide Movement: +)([\w -]+\w+)(   *)', info)
tide_movement = tide_movement_match.group(2)
tide_movement

'falling'

In [10]:
tide_state_match = re.search(r'(Tide State: +)([\w -]+\w+)(   *)', info)
tide_state = tide_state_match.group(2)
tide_state

'high'

In [11]:
sky_match = re.search(r'(Sky: +)([\w -]+\w+)(   *)', info)
sky = sky_match.group(2)
sky

'clear'

In [12]:
sea_state_match = re.search(r'(Sea State: +)([\w -]+\w+)(   *)', info)
sea_state = sea_state_match.group(2)
sea_state

'calm - ripple - chop'

In [13]:
date_match = re.search(r'(Date:\s+)([\w -,]+\w+)(   *)', info)
date = date_match.group(2)
date

'January 9, 2022'

In [14]:
start_match = re.search(r'(Start:\s+)(\w+:\w+)(   *)', info_1)
start = start_match.group(2)
start

'08:30'

In [15]:
finish_match = re.search(r'(Finish:\s+)(\w+:\w+)(   *)', info_1)
finish = finish_match.group(2)
finish

'2:00'

In [16]:
precip_match = re.search(r'(Precip: +)([\w -]+\w+)(   *)', info)
precip = precip_match.group(2)
precip

'none'

In [40]:
# get the header row
idx = df[df[0].str.contains(r'Species: +\w+', regex=True, na=False)].index
idx[0]

4

In [73]:
df1 = pd.read_excel(file_path,  sheet_name='Jul', header=idx[0])
columns = df1.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       76 non-null     object
 1   1       75 non-null     object
 2   2       6 non-null      object
 3   3       10 non-null     object
 4   4       17 non-null     object
 5   5       11 non-null     object
 6   6       19 non-null     object
 7   7       14 non-null     object
 8   8       29 non-null     object
 9   9       24 non-null     object
 10  10      29 non-null     object
 11  11      24 non-null     object
 12  12      8 non-null      object
 13  13      29 non-null     object
 14  14      40 non-null     object
 15  15      17 non-null     object
 16  16      19 non-null     object
 17  17      20 non-null     object
 18  18      15 non-null     object
 19  19      34 non-null     object
 20  20      16 non-null     object
 21  21      22 non-null     object
dtypes: object(22)
memory usage: 

In [74]:

df2 = df1.dropna(subset=[columns[0]])
df2 = df2.drop(columns='Total')
df2 = df2.fillna(0)
df2 = df2.sort_values([columns[0]])
df2 = df2.set_index(columns[0])
df2.index.name=None
df2 = df2.drop('Totals')


In [46]:
# transpose to long form
df4 = pd.DataFrame()
for species in df2.index:
    species_count = df2.loc[species].T
    species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date': '2022-01', 'species': species})
    df4 = pd.concat([df4, species_df])
df4

,id,count,date,species
0,OE,0,2022-01,Accipiter sp.
1,WD,0,2022-01,Accipiter sp.
2,SR,0,2022-01,Accipiter sp.
3,TD1,0,2022-01,Accipiter sp.
4,TD2,0,2022-01,Accipiter sp.
...,...,...,...,...
15,SA,1,2022-01,Winter Wren (Pacific)
16,IM,0,2022-01,Winter Wren (Pacific)
17,MC1,0,2022-01,Winter Wren (Pacific)
18,MC2,0,2022-01,Winter Wren (Pacific)


In [41]:
# Total species
total_count = df2.gt(0).sum(axis=0)
df3 = pd.DataFrame({'id': total_count.index, 'counts': total_count.values, 'date':'2022-01', 'species': 'Total Species'})
df3

,id,counts,date,species
0,OE,6,2022-01,Total Species
1,WD,4,2022-01,Total Species
2,SR,7,2022-01,Total Species
3,TD1,5,2022-01,Total Species
4,TD2,8,2022-01,Total Species
5,TD3,3,2022-01,Total Species
6,EF1,11,2022-01,Total Species
7,EF2,5,2022-01,Total Species
8,CC,8,2022-01,Total Species
9,CS,12,2022-01,Total Species


# Load Data from excel sheets

In [45]:
def read_excel_file(year):

    excel_df = pd.DataFrame()
    file_path = os.path.join(r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly', f'embc{year}.xlsx')
    
    for month in months:
        df = pd.read_excel(file_path,  sheet_name=month, header=None)
        # idx = df[df[0].str.contains(r'Species: +\w+', regex=True, na=False)].index
        #find header row
        header_idx = df[df[2].str.contains(r'OE', regex=True, na=False)].index
        df1 = pd.read_excel(file_path,  sheet_name=month, header=header_idx[0])
        columns = df1.columns
        #find extra header rows and remove
        df1[columns[2]] = df1[columns[2]].astype('str')
        extra_header_idx = df1[df1[columns[2]].str.contains(r'OE', regex=True, na=False)].index
        if len(extra_header_idx) > 0:
            for header_row_idx in extra_header_idx:
                df1 = df1.drop(header_row_idx)
        
        df2 = df1.dropna(subset=[columns[0]])
        df2 = df2.drop(columns='Total')
        df2 = df2.fillna(0)
        df2 = df2.sort_values([columns[0]])
        total_idx = df2[df2[columns[0]].str.contains(r'Totals|Overall.*', regex=True, na=False)].index
        # df2.loc[total_idx,columns[0]] = 'Total Bird Count'
        df2 = df2.drop(total_idx)

        # drop species that start with asterisk
        asterisk_idx = df2[df2[columns[0]].str.contains(r'^\*', regex=True, na=False)].index
        df2 = df2.drop(asterisk_idx)

        df2 = df2.set_index(columns[0])
        df2.index.name=None

        # individual species count
        for species in df2.index:
            species_count = df2.loc[species].T
            species_count = pd.to_numeric(species_count, errors='coerce').fillna(0)
            species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date':f'{month}-{year}', 'year': year, 'month': month,'species': species})
            excel_df = pd.concat([excel_df, species_df])

    excel_df['count'] = excel_df['count'].astype('int64')
    return excel_df

In [46]:
excel_df = pd.DataFrame()
for year in range(2010,2023):
    excel_year_df = read_excel_file(year)
    excel_df = pd.concat([excel_df, excel_year_df])


In [47]:
# Replacements Dictionary to correct spelling from excel sheets
replacements = {
    'Accipter sp.': 'Accipiter sp.',
    'American Crow x Northwestern Crow': 'American Crow',
    'American Robbin': 'American Robin',
    'American Wigeion': 'American Wigeon',
    'Anna’s Humming Bird': "Anna's Hummingbird",
    'Anna’s Hummingbird': "Anna's Hummingbird",
    'Back Swift': 'Black Swift',
    'Barrows Goldeneye': "Barrow's Goldeneye",
    "Barrrow's Goldeneye": "Barrow's Goldeneye",
    'Black-throated Grey Warbler': 'Black-throated Gray Warbler',
    'Blue Grouse (Sooty)': 'Sooty Grouse',
    'Bustit': 'Bushtit',
    'California (Western) Scrub-Jay': 'California Scrub-Jay',
    'Cedar Wagwing': 'Cedar Waxwing',
    'Chestnut-back Chickadee': 'Chestnut-backed Chickadee',
    ' Dark-eyed Junco': 'Dark-eyed Junco',
    'Doubel-crested Cormorant': 'Double-crested Cormorant',
    'Doule-crested Cormorant': 'Double-crested Cormorant',
    'Eurasian Collar-Dove': 'Eurasian Collared Dove',
    'Eurasian Collared-Dove': 'Eurasian Collared Dove',
    'Flycatcher, sp.': 'Flycatcher sp.',
    'Glaucous-winged x Herring Gull': 'Glaucous-winged Gull',
    'Gul sp.': 'Gull sp.',
    'Gull Hybrid': 'Gull, Hybrid',
    'Gull, Glaucous-winged x Western': 'Glaucous-winged Gull',
    'Gull, Western x Glaucous-wing': 'Glaucous-winged Gull',
    'Gull, Western x Glaucous-winged': 'Glaucous-winged Gull',
    'Gull,Glaucous-winged x Western': 'Glaucous-winged Gull',
    'Hawk Species': 'Hawk sp.',
    'Hawk, sp.': 'Hawk sp.',
    "Lincoln's Sparow": "Lincoln's Sparrow",
    'Lincoln’s Sparrow': "Lincoln's Sparrow",
    'Marsh Wren ': 'Marsh Wren',
    'Norther Harrier': 'Northern Harrier',
    'Northern Pygmy-owl': 'Northern Pygmy Owl',
    'Northern Pygmy-Owl': 'Northern Pygmy Owl',
    'Northwestern Crow': 'American Crow',
    'Nthn Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winges Swallow': 'Northern Rough-winged Swallow',
    'Olive-sided Flcatcher': 'Olive-sided Flycatcher',
    'Pacific-Slope Flycatcher': 'Pacific-slope Flycatcher',
    'Passerine sp': 'Passerine sp.',
    'Pelgagic Cormorant': 'Pelagic Cormorant',
    'Peregrin Falcon': 'Peregrine Falcon',
    'Peregrine': 'Peregrine Falcon',
    'Peregrine Falacon': 'Peregrine Falcon',
    'Pied -billed Grebe': 'Pied-billed Grebe',
    'Pied Bil Grebe': 'Pied-billed Grebe',
    'Pileated Wodpecker': 'Pileated Woodpecker',
    'Red Throated Loon': 'Red-throated Loon',
    'Red-tailed hawk': 'Red-tailed Hawk',
    'Ring-biled Gull': 'Ring-billed Gull',
    'Rock Dove': 'Pigeon',
    'Rock Dove (Pigeon)': 'Pigeon',
    'Rock Dove (Rock Pigeon)': 'Pigeon',
    'Rock Pigeon': 'Pigeon',
    'Ruby-crowned Kinglety': 'Ruby-crowned Kinglet',
    'Savannah': 'Savannah Sparrow',
    'Scaup Sp.': 'Scaup sp.',
    'Scaup sp': 'Scaup sp.',
    'Semi-palmated Sandpiper':  'Semipalmated Sandpiper',
    'Sharp-Shinned Hawk': 'Sharp-shinned Hawk', 
    'Sharp-shinned Hawk*': 'Sharp-shinned Hawk',
    'Short-blled Gull': 'Short-billed Gull',
    'Sooty (Blue) Grouse': 'Sooty Grouse',
    'Spotted Sandiper': 'Spotted Sandpiper',
    "Stellar's Jay": "Steller's Jay",
    'Steller’s Jay': "Steller's Jay",
    'Stellar’s Jay': "Steller's Jay",
    'Swainson’s Thrush': "Swainson's Thrush",
    'Townsend’s Warbler': "Townsend's Warbler",
    "Vaux' Swift": "Vaux's Swift",
    'Vaux’s Swift': "Vaux's Swift",
    'White-Winged Crossbill': 'White-winged Crossbill',
    'Winter (Pacific) Wren': 'Pacific Wren',
    'Winter Wren': 'Pacific Wren',
    'Winter Wren (Pacific Wren)': 'Pacific Wren',
    'Winter Wren (Pacific)': 'Pacific Wren',
    "Wison's Warbler": "Wilson's Warbler",
    'Woodpecker sp': 'Woodpecker sp.',
    'chickadee sp.': 'Chickadee sp.',
    'finch sp.': 'Finch sp.',
    'gull sp.': 'Gull sp.',
    'hummingbird sp.': 'Hummingbird sp.',
    'sparrow sp.': 'Sparrow sp.',
    'swallow sp.': 'Swallow sp.',
}

In [48]:
df = excel_df.replace(replacements)

In [49]:
spell_checked_df = df.replace(replacements)

In [10]:
df[(df['species']=='Dark-eyed Junco') & (df['year']==2013)]

,id,count,date,year,month,species
0,OE,0,Jan-2013,2013,Jan,Dark-eyed Junco
1,WD,0,Jan-2013,2013,Jan,Dark-eyed Junco
2,SR,0,Jan-2013,2013,Jan,Dark-eyed Junco
3,TD1,0,Jan-2013,2013,Jan,Dark-eyed Junco
4,TD2,0,Jan-2013,2013,Jan,Dark-eyed Junco
5,TD3,0,Jan-2013,2013,Jan,Dark-eyed Junco
6,EF1,0,Jan-2013,2013,Jan,Dark-eyed Junco
7,EF2,0,Jan-2013,2013,Jan,Dark-eyed Junco
8,CC,10,Jan-2013,2013,Jan,Dark-eyed Junco
9,CS,8,Jan-2013,2013,Jan,Dark-eyed Junco


In [50]:
# area total bird count 
areas_total_bird_count_df = spell_checked_df.groupby(['id','year','month', 'date']).sum('count').reset_index()
areas_total_bird_count_df['species'] = 'Total Bird Count'
areas_total_added_df = pd.concat([spell_checked_df, areas_total_bird_count_df])

all_area_total_bird = areas_total_added_df.groupby(['year','month', 'date', 'species']).sum('count').reset_index()
all_area_total_bird['id'] = 'ALL'

all_area_total_added_df = pd.concat([areas_total_added_df, all_area_total_bird])

In [17]:
all_area_total_added_df[(all_area_total_added_df['species']=='Dark-eyed Junco') & (all_area_total_added_df['year']==2013)]

,id,count,date,year,month,species
0,OE,0,Jan-2013,2013,Jan,Dark-eyed Junco
1,WD,0,Jan-2013,2013,Jan,Dark-eyed Junco
2,SR,0,Jan-2013,2013,Jan,Dark-eyed Junco
3,TD1,0,Jan-2013,2013,Jan,Dark-eyed Junco
4,TD2,0,Jan-2013,2013,Jan,Dark-eyed Junco
5,TD3,0,Jan-2013,2013,Jan,Dark-eyed Junco
6,EF1,0,Jan-2013,2013,Jan,Dark-eyed Junco
7,EF2,0,Jan-2013,2013,Jan,Dark-eyed Junco
8,CC,10,Jan-2013,2013,Jan,Dark-eyed Junco
9,CS,8,Jan-2013,2013,Jan,Dark-eyed Junco


In [51]:

# sum species in each area, making sure to remove "Total Bird Count" from totals
all_area_species = all_area_total_added_df[all_area_total_added_df['species']!='Total Bird Count']
all_area_total_species = all_area_species.groupby(['id', 'date', 'year', 'month']).apply(lambda x: x['count'].gt(0).sum(axis=0)).reset_index(name='count')
all_area_total_species['species'] = 'Total Species Count'


areas_and_species_totals_added_df = pd.concat([all_area_total_added_df, all_area_total_species])


In [20]:
areas_and_species_totals_added_df[(areas_and_species_totals_added_df['species']=='Dark-eyed Junco') & (areas_and_species_totals_added_df['year']==2013)]

,id,count,date,year,month,species
0,OE,0,Jan-2013,2013,Jan,Dark-eyed Junco
1,WD,0,Jan-2013,2013,Jan,Dark-eyed Junco
2,SR,0,Jan-2013,2013,Jan,Dark-eyed Junco
3,TD1,0,Jan-2013,2013,Jan,Dark-eyed Junco
4,TD2,0,Jan-2013,2013,Jan,Dark-eyed Junco
5,TD3,0,Jan-2013,2013,Jan,Dark-eyed Junco
6,EF1,0,Jan-2013,2013,Jan,Dark-eyed Junco
7,EF2,0,Jan-2013,2013,Jan,Dark-eyed Junco
8,CC,10,Jan-2013,2013,Jan,Dark-eyed Junco
9,CS,8,Jan-2013,2013,Jan,Dark-eyed Junco


In [23]:
# test_final3[test_final3['month'].isnull()]
len(areas_and_species_totals_added_df.species.unique())

251

In [52]:
df_loaded = areas_and_species_totals_added_df
df_loaded['month'] = pd.Categorical(df_loaded['month'], months)
df_loaded = df_loaded.sort_values(['year', 'month','species']).reset_index(drop=True)
# df_loaded

In [53]:
df_loaded[(df_loaded['year']==2013) & (df_loaded['id']=='ALL') & (df_loaded['species']=='Dark-eyed Junco')]


,id,count,date,year,month,species
48583,ALL,168,Jan-2013,2013,Jan,Dark-eyed Junco
49885,ALL,88,Feb-2013,2013,Feb,Dark-eyed Junco
51103,ALL,115,Mar-2013,2013,Mar,Dark-eyed Junco
52447,ALL,95,Apr-2013,2013,Apr,Dark-eyed Junco
55764,ALL,2,Jun-2013,2013,Jun,Dark-eyed Junco
60993,ALL,33,Oct-2013,2013,Oct,Dark-eyed Junco
62315,ALL,53,Nov-2013,2013,Nov,Dark-eyed Junco
63428,ALL,74,Dec-2013,2013,Dec,Dark-eyed Junco


In [30]:
year_all_total

,id,count,date,year,month,species


In [81]:
zero_counts_added = df_loaded.copy()
for year in range(2010,2023):
    year_all_total = df_loaded[(df_loaded['year']==year) & (df_loaded['id']=='ALL')]
    # print(year_all_total)
    # build dataframe with no counts for all species in all months of the year
    merge_template = pd.DataFrame()
    for specie in year_all_total.species.unique():
        temp = pd.DataFrame({'id': 'ALL', 'date': [f'{month}-{year}' for month in months], 'year': year, 'month':months, 'species': specie})
        if specie == 'Common Loon':
            print(temp)
        merge_template = pd.concat([merge_template, temp])
        
    # merge with the count and fill na with 0
    merge_df = merge_template.merge(year_all_total, how='left', on=['id', 'date', 'year', 'month', 'species']).fillna(0)
    # print(merge_df['species'])
    zero_counts = merge_df[merge_df['count']==0]
    zero_counts_added = pd.concat([zero_counts_added, zero_counts])


     id      date  year month      species
0   ALL  Jan-2010  2010   Jan  Common Loon
1   ALL  Feb-2010  2010   Feb  Common Loon
2   ALL  Mar-2010  2010   Mar  Common Loon
3   ALL  Apr-2010  2010   Apr  Common Loon
4   ALL  May-2010  2010   May  Common Loon
5   ALL  Jun-2010  2010   Jun  Common Loon
6   ALL  Jul-2010  2010   Jul  Common Loon
7   ALL  Aug-2010  2010   Aug  Common Loon
8   ALL  Sep-2010  2010   Sep  Common Loon
9   ALL  Oct-2010  2010   Oct  Common Loon
10  ALL  Nov-2010  2010   Nov  Common Loon
11  ALL  Dec-2010  2010   Dec  Common Loon
     id      date  year month      species
0   ALL  Jan-2011  2011   Jan  Common Loon
1   ALL  Feb-2011  2011   Feb  Common Loon
2   ALL  Mar-2011  2011   Mar  Common Loon
3   ALL  Apr-2011  2011   Apr  Common Loon
4   ALL  May-2011  2011   May  Common Loon
5   ALL  Jun-2011  2011   Jun  Common Loon
6   ALL  Jul-2011  2011   Jul  Common Loon
7   ALL  Aug-2011  2011   Aug  Common Loon
8   ALL  Sep-2011  2011   Sep  Common Loon
9   ALL  Oc

In [82]:
zero_counts_added[(zero_counts_added['species']=='Common Loon') & (zero_counts_added['year']==2022) & (zero_counts_added['id']=='ALL')]

,id,count,date,year,month,species
195728,ALL,18.0,Apr-2022,2022,Apr,Common Loon
199424,ALL,1.0,Jun-2022,2022,Jun,Common Loon
201124,ALL,1.0,Jul-2022,2022,Jul,Common Loon
206101,ALL,3.0,Nov-2022,2022,Nov,Common Loon
207277,ALL,2.0,Dec-2022,2022,Dec,Common Loon
828,ALL,0.0,Jan-2022,2022,Jan,Common Loon
829,ALL,0.0,Feb-2022,2022,Feb,Common Loon
830,ALL,0.0,Mar-2022,2022,Mar,Common Loon
832,ALL,0.0,May-2022,2022,May,Common Loon
835,ALL,0.0,Aug-2022,2022,Aug,Common Loon


In [83]:
df = zero_counts_added
df['month'] = pd.Categorical(df['month'], months)
df = df.sort_values(['year', 'month','species']).reset_index(drop=True)
df['count'] = df['count'].astype('int64')

In [84]:
len(df.species.unique())

250

In [5]:
# Stats for df
def get_stats_df(df):
    stats_df = df[df['id']=='ALL']
    stats_df = stats_df.groupby(
        ['month', 'species']
        ).agg(
            mean=('count', np.mean),
            median=('count', np.median),
            std=('count', np.std),
            min=('count', np.min),
            max=('count', np.max)
        ).reset_index()

    stats_df['month'] = pd.Categorical(stats_df['month'], months)
    stats_df = stats_df.sort_values(['month','species']).reset_index(drop=True)
    
    return stats_df

In [86]:
get_stats_df(df[(df['species']== "Dark-eyed Junco") & (df['id']=='ALL')])

,month,species,mean,median,std,min,max
0,Jan,Dark-eyed Junco,135.923077,134.0,56.301956,52,217
1,Feb,Dark-eyed Junco,84.461538,71.0,32.255789,47,156
2,Mar,Dark-eyed Junco,98.615385,102.0,51.640647,16,227
3,Apr,Dark-eyed Junco,106.538462,95.0,97.978412,4,329
4,May,Dark-eyed Junco,10.384615,1.0,21.696922,0,70
5,Jun,Dark-eyed Junco,0.923077,1.0,1.382120,0,5
6,Jul,Dark-eyed Junco,0.384615,0.0,0.767948,0,2
7,Aug,Dark-eyed Junco,0.615385,0.0,1.192928,0,4
8,Sep,Dark-eyed Junco,1.307692,1.0,1.493576,0,4
9,Oct,Dark-eyed Junco,52.538462,51.0,26.244413,14,115


# Save data

In [87]:
df.to_csv('count_data.csv')

# App

read areas and mapbox token

In [2]:
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\data\areas.json"
) as areas_file:
    areas = json.load(areas_file)
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\.mapbox_token"
) as token_file:
    token = token_file.read()



Get Data from csv

In [3]:
df = pd.read_csv(r'C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\bird_count\data\count_data.csv', index_col=0)

In [8]:
app = JupyterDash(__name__)

app.layout = html.Div(
    id="app-container",
    children=[
        html.Div(
            id="sidebar-container",
            children=[
                html.H1("Squamish Monthly Bird Count"),
                dcc.Tabs(
                    id="tabs",
                    value="tab-graph",
                    children=[
                        dcc.Tab(label="Graph", value="tab-graph"),
                        dcc.Tab(label="Map", value="tab-map"),
                    ],
                ),
                html.P("Select Species:"),
                        dcc.Dropdown(
                            id="species-dropdown",
                            options=sorted(df["species"].unique()),
                            value="Total Species Count",
                            clearable=False,
                        ),
                html.Div(
                    id='sidebar-content',
                    ),
            ],
        ),
        html.Div(
            id="content-container",
        ),
    ],
)

@app.callback(
    Output('sidebar-content', 'children'),
    Input('tabs', 'value')
)
def render_sidebar_content(tab):
    if tab == 'tab-map':
        return []
    elif tab == 'tab-graph':
        return [
            html.P("Line Shape:"),
            dcc.RadioItems(
                ['spline', 'linear'],
                'spline',
                id='line-shape-radio',
                inline=True
            ),
        ]


@app.callback(
    Output('content-container', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-map':
        return dcc.Graph(
            id="count-map",
            config=dict(responsive=True),
        )
    elif tab == 'tab-graph':
        return dcc.Graph(
            id="count-graph",
            config=dict(responsive=True),
        )

@app.callback(
    Output("count-map", "figure"), 
    Input("species-dropdown", "value")
)
def update_map(species):
    dff = df[df["species"] == species]
    
    fig = px.choropleth_mapbox(
        dff,
        geojson=areas,
        locations="id",
        featureidkey="properties.id",
        color="count",
        color_continuous_scale="Purples",
        range_color=(0, dff["count"].max()),
        zoom=12.5,
        center={"lat": 49.7, "lon": -123.15},
        opacity=0.5,
        labels={"count": "Count", "id": "Area"},
        animation_frame="date",
        template="plotly_dark",
    )
    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
        mapbox_accesstoken=token,
        mapbox_style="satellite-streets",
    )
    if fig["layout"]["updatemenus"]:
        fig["layout"]["updatemenus"][0]["pad"] = dict(r=20, t=25)
        fig["layout"]["sliders"][0]["pad"] = dict(r=0, t=0, b=20)

    return fig

@app.callback(
    Output("count-graph", "figure"), 
    Input("species-dropdown", "value"),
    Input("line-shape-radio", "value")
)
def update_graph(species, line_shape):
    dff = df[(df['species']== species) & (df['id']=='ALL')]
    stats_df = get_stats_df(dff)

    fig = px.line(
        dff,
        x='month',
        y='count',
        category_orders= {'month': months},
        color="year",
        color_discrete_sequence=px.colors.qualitative.Light24,
        template="plotly_dark",
        markers=True,
        line_shape=line_shape,
    )

    # Average
    fig.add_trace(
        go.Scatter(
            x=stats_df['month'],
            y=stats_df['mean'], 
            mode="lines",
            line_shape=line_shape,
            name='Average',
            line={'width':4, 'color':'white'}
        )
    )
    
    # Standard deviation line
    average_plus_std = list(stats_df['mean']+stats_df['std'])
    average_minus_std = list(stats_df['mean']-stats_df['std'])
    rev_average_minus_std = average_minus_std[::-1]
    rev_average_minus_std = [x if x > 0 else 0 for x in rev_average_minus_std]

    fig.add_trace(
        go.Scatter(
            x=months+months[::-1],
            y=average_plus_std+rev_average_minus_std,
            fill='toself',
            fillcolor='rgba(255,255,255,0.3)',
            line_color='rgba(255,255,255,0)',
            mode="lines",
            line_shape=line_shape,
            name='Standard Deviation',
            line={'width':4, 'color':'white'}
        )
    )

    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
    )

    fig.data = fig.data[::-1]
    return fig



if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
